In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = ""

import numpy as np
import keras
from keras.models import Model, load_model
import midi
import cv2

dir_name = 'History/'
sub_dir_name = 'e2000/'

# print "Loading Keras..."
# print "Keras Version: " + keras.__version__

from keras import backend as K

K.set_image_data_format('channels_first')

# print "Loading Encoder..."
model = load_model(dir_name + 'model.h5')
enc = K.function([model.get_layer('encoder').input, K.learning_phase()],
                 [model.layers[-1].output])
enc_model = Model(inputs=model.input, outputs=model.get_layer('pre_encoder').output)

# print "Loading Statistics..."
means = np.load(dir_name + sub_dir_name + 'means.npy')
evals = np.load(dir_name + sub_dir_name + 'evals.npy')
evecs = np.load(dir_name + sub_dir_name + 'evecs.npy')
stds = np.load(dir_name + sub_dir_name + 'stds.npy')

# print "Loading Songs..."
y_samples = np.load('samples.npy')
y_lengths = np.load('lengths.npy')


Using TensorFlow backend.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



In [2]:
import time
print (time.strftime("Live_%H:%M:%S.mid"))

Live_09:32:56.mid


In [3]:
cur_params = np.zeros((120,))
cur_params[1] = -5
note_thresh = 10

In [4]:
x = means + np.dot(cur_params * evals, evecs)
x = np.expand_dims(x, axis=0)

In [5]:
y = enc([x, 0])[0][0]
cur_notes = (y * 255.0).astype(np.uint8)

In [5]:
midi.samples_to_midi(cur_notes, 'live2.mid', 16, note_thresh)

In [6]:
def generate():
    global cur_notes
    global note_thresh
    filename = time.strftime("GeneratedAudio/Live_%H:%M:%S.mid")
    midi.samples_to_midi(cur_notes, filename, 16, note_thresh)
    print(f"Generated: {filename}")

In [ ]:
import tkinter as tk
from PIL import Image, ImageTk
from functools import partial

root = tk.Tk()
root.title("MIDI generate")
vector = [0 for _ in range(20)]

def update_vector(j, val):
    global vector
    global cur_notes
    global canvass
    global imgs
    global note_thresh
    

#     cur_params = np.zeros((120,))
    cur_params[j] = float(val)
    x = means + np.dot(cur_params * evals, evecs)
    x = np.expand_dims(x, axis=0)
    y = enc([x, 0])[0][0]
    cur_notes = (y * 255.0).astype(np.uint8)
    imgs = []
    for i in range(16):
        
        cur_note_for_image = np.copy(cur_notes[i])
        cur_note_for_image[cur_note_for_image<=note_thresh] = 0
        new = cv2.applyColorMap(cur_note_for_image, cv2.COLORMAP_HOT)                                                                                                                                                    
        rgb_img = cv2.resize(new, (int(96 * 1.5), int(96 * 1.5)), interpolation=cv2.INTER_NEAREST) 

        b,g,r = cv2.split(rgb_img)
        rgb_img = cv2.merge((r,g,b))
        imgs.append(ImageTk.PhotoImage(image=Image.fromarray(rgb_img)))
        canvass[i].configure(image=imgs[-1])
        canvass[i].image = imgs[-1]
    
def update_thresh(val):
    global note_thresh
    note_thresh = float(val)
    imgs = []
    for i in range(16):
        cur_note_for_image = np.copy(cur_notes[i])
                    
        cur_note_for_image[cur_note_for_image<=note_thresh] = 0
        new = cv2.applyColorMap(cur_note_for_image, cv2.COLORMAP_HOT)                                                                                                                                                    
        rgb_img = cv2.resize(new, (int(96 * 1.5), int(96 * 1.5)), interpolation=cv2.INTER_NEAREST) 

        b,g,r = cv2.split(rgb_img)
        rgb_img = cv2.merge((r,g,b))
        imgs.append(ImageTk.PhotoImage(image=Image.fromarray(rgb_img)))
        canvass[i].configure(image=imgs[-1])
        canvass[i].image = imgs[-1]
    
def print_type(self, event): 
    print(event.type) 

sliders = [
    tk.Scale(root, from_=-5, to_=5, resolution=0.1, orient=tk.VERTICAL, command=partial(update_vector, i), showvalue=0)
    for i in range(32)
]
for i, slider in enumerate(sliders):
    slider.grid(column=i, row = 0)
    
    
canvass = []
imgs = []


for i in range(16):
    new = cv2.applyColorMap(cur_notes[i], cv2.COLORMAP_HOT)                                                                                                                                                    
    rgb_img = cv2.resize(new, (int(96 * 1.5), int(96 * 1.5)), interpolation=cv2.INTER_NEAREST) 
    
    b,g,r = cv2.split(rgb_img)
    rgb_img = cv2.merge((r,g,b))
    imgs.append(ImageTk.PhotoImage(image=Image.fromarray(rgb_img)))
    canvass.append(tk.Label(root, image=imgs[-1]))
    canvass[-1].grid(column=i*4 - int(i/8)*8*4, row=1 + int(i/8), columnspan=4)

tk.Button(root, text="Generate", command=generate).grid(column=1, row=3, columnspan=2)
tk.Scale(root, from_=200, to_=10, resolution=1, orient=tk.HORIZONTAL, 
         showvalue=1, length=300, command=update_thresh).grid(column=3, row=3, columnspan=9)
root.mainloop()


Generated: GeneratedAudio/Live_11:29:41.mid
Generated: GeneratedAudio/Live_11:29:45.mid
